# Project Supermarket Introduction

## Project Goal

Write a program that simulates and visualizes customer behavior in a supermarket. (eg. https://github.com/win-times-win/supermarket_simulation) and use this to
- Compare the simulated with the real data
- Visualize the simulation (increases understandability)
- Formulate business questions and answer them using the simulation.
   - How many checkout-counters have to be open at the same time to make sure the queue does not get longer than x people? or
   - How many customers can be allowed to enter a supermarket at the same time so that no area is visited by more than y customers at the same time?

## Subtasks

- Analyze the data and calculate the transition probability matrix for the Markov-Chain
- Implement a Markov-Chain based data generator
- Implement customer and supermarket classes (object oriented programing)
- Simulate customer behavior and the supermarket environment (Monte Carlo Simulation)
- Visualize the simulated customer movement
- Answer your questions

## Concepts

1. Monte Carlo Methods
2. Markov Chains
3. Object-oriented programming (Classes)
4. Programming techniques (Program Design, Structuring Programs)

Let's talk about more about the first two of these concepts

### 1) Monte Carlo Methods

A class of computational algorithms that rely on repeated random sampling to obtain numerical results.

For example, in order to answer the question about the necessary number of open checkout-counters we can (and will) randomly draw samples from the probability distribution given by the customer behavior.

The following question arises: **How do we model customer behavior?** To answer this question we introduce the concept of Markov Chains.

### 2) Markov Chains

- A Markov Chain is a stochastic process that has the Markov property.
- "Markov chains, named after Andrey Markov, are mathematical systems that hop from one "state" (a situation or set of values) to another." https://setosa.io/blog/2014/07/26/markov-chains/index.html

#### Markov Property

A stochastic process has the Markov Property, if knowledge of the current state is all that is necessary to determine the probability distribution of future states. The stochastic process is said to be memoryless.

#### Assumptions

1. Markov Property: Knowledge of the current state is all that is necessary to determine the probability distribution of future states.
- Finite State Space: `states = []`
- No Hidden States: all states are known and observable
- Discrete time: time is measured in discrete steps
- Time-homogenous: transition probabilities do not change with time

#### Characteristics

A Markov Chain can be characterized by three elements:

- State space (all possible states)
- Initial state or initial probability distribution
- Transition probabilities

In [2]:
!pip install pandas

     |████████████████████████████████| 10.0 MB 1.2 MB/s eta 0:00:01    |██████▎                         | 2.0 MB 388 kB/s eta 0:00:21
  Using cached pytz-2019.3-py2.py3-none-any.whl (509 kB)


In [3]:
import random
import pandas as pd
import numpy as np

![](./markov_weather.png)

In [4]:
# Let's store that information
weather = ['snow', 'snow', 'sun', 'snow', 'snow', 'sun', 'snow', 'sun', 'sun']
weather

['snow', 'snow', 'sun', 'snow', 'snow', 'sun', 'snow', 'sun', 'sun']

## How can we calculate the transition matrix?

The transition matrix $P$ has the element $p_{ij}$, with rows $i$ and columns $j$, such that:

$$
p_{ij} = P(Y_t = y_j | Y_{t-1} = y_i)
$$

For example $p_{0,1} = p_{snow, sun}$ is the probability of a sunny day when it was snowing the day before. 

In [5]:
weather = pd.DataFrame({'time': range(1,len(weather)+1), 'weather': weather})
weather

,time,weather
0,1,snow
1,2,snow
2,3,sun
3,4,snow
4,5,snow
5,6,sun
6,7,snow
7,8,sun
8,9,sun


In [6]:
weather['transition'] = weather['weather'].shift(-1)
weather

,time,weather,transition
0,1,snow,snow
1,2,snow,sun
2,3,sun,snow
3,4,snow,snow
4,5,snow,sun
5,6,sun,snow
6,7,snow,sun
7,8,sun,sun
8,9,sun,NaN


In [7]:
weather.groupby(['weather', 'transition']).count()

time
weather transition      
snow    snow           2
        sun            3
sun     snow           2
        sun            1

In [8]:
weather.groupby('weather')['transition'].value_counts(normalize=True)

weather  transition
snow     sun           0.600000
         snow          0.400000
sun      snow          0.666667
         sun           0.333333
Name: transition, dtype: float64

In [9]:
P = pd.crosstab(weather['weather'], weather['transition'], normalize=0)

## Creating a state diagram with [`pygraphviz`](http://pygraphviz.github.io/documentation/pygraphviz-1.5/)

In [10]:
import pygraphviz as pgv

states = ['snow', 'sun']
G = pgv.AGraph(directed=True)
for state_from in states:
    for state_to in states:
        G.add_edge(state_from, state_to, label=np.round(P.loc[state_from, state_to],2))
G.draw('transition.png', prog='dot')

ModuleNotFoundError: No module named 'pygraphviz'

## So we have

$$
P(Y_{t+1}=0| Y_{t}=0) = 0.4
$$
$$
P(Y_{t+1}=0| Y_{t}=1) = 0.67
$$
$$
P(Y_{t+1}=1| Y_{t}=0) = 0.60
$$
$$
P(Y_{t+1}=1| Y_{t}=1) = 0.33
$$

What is the probability of a third day of sun given that the two previous days where sunny as well (Markov property!)?

$$
P(Y_{3}=1|Y_{2}=1, Y_{1}=1) = P(Y_{3}=1|Y_{2}=1) = 0.33
$$

What is the probability of a sunny third day given that the first day was sunny and we know nothing about the second day? 

$$
P(Y_{3}=1|Y_{1}=1) = P(Y_{3}=1| Y_{2}=1)P(Y_{2}=1|Y_{1}=1) + P(Y_{3}=1| Y_{2}=0)P(Y_{2}=0|Y_{1}=1) = 0.33*0.33 + 0.6*0.67 = 0.51
$$

In general we can write:

$$
P(Y_{t+1} = y_{t+1} | Y_{1}=y_{1}) = P(Y_{0}=y_{0})P^t
$$

where $P(Y_{0})$ is the given initial distribution.

In [11]:
# next state probabilities given the initial state
P = P.values
P

array([[0.4       , 0.6       ],
       [0.66666667, 0.33333333]])

In [30]:
P

array([[0.4       , 0.6       ],
       [0.66666667, 0.33333333]])

In [28]:
one_day_ahead

array([0.66666667, 0.33333333])

In [31]:
val1 = (0.4 * 0.66667) + (0.66667 * 0.3333) 
val2 = (0.6 * 0.66667) + (0.3333 * 0.33333)
val2

0.511100889

In [ ]:
pd.Series

In [ ]:
dataframe = rows = # of aisles
            cols = # of aisles
            values = #probability of moving from row position - col position
dairy    = [0,1] #these have to sum to one
checkout = [0,0]

In [25]:
two_day_ahead

array([0.48888889, 0.51111111])

In [17]:
current_state = np.array([0, 1])
one_day_ahead = current_state.dot(P)
one_day_ahead

array([0.66666667, 0.33333333])

In [19]:
one_day_ahead.shape

(2,)

In [13]:
two_day_ahead = one_day_ahead.dot(P)

In [14]:
two_day_ahead

array([0.48888889, 0.51111111])

In [25]:
# state probabilities two days ahead P^2
P.dot(P)

array([[0.56      , 0.44      ],
       [0.48888889, 0.51111111]])

In [23]:
# stationary distribution
P.dot(P).dot(P).dot(P).dot(P).dot(P).dot(P)

array([[0.52627037, 0.47372963],
       [0.52636626, 0.47363374]])

#### How can we characterize the Markov Chain in the supermarket project?

In [ ]:
initial_state
transition_probabilities
states = ['fruit', 'dairy', 'drinks', 'spices', 'checkout']